# First lets process the stylish data

In [1]:
import os
import pandas as pd

In [8]:
eps_paths = []

for f in os.listdir('/l/disk1/awstebas/data/TTS/speaker-adriana/'):
    if("eps_neutro" in f or "eps_animado" in f or "eps_rispido" in f or 'eps_acolhedor' in f):
        if(f[:4] != "sint"):
            eps_paths.append(f)

In [9]:
# The code above should result in this cell output. Maybe you should change the above code
eps_paths

['eps_neutro_ani',
 'eps_rispido_gen',
 'eps_rispido_acorf',
 'eps_neutro_gen',
 'eps_animado_ani',
 'eps_acolhedor_rf',
 'eps_acolhedor_gen',
 'eps_neutro_rf',
 'eps_neutro_aco',
 'eps_acolhedor_aco',
 'eps_animado_aco',
 'eps_rispido_ris',
 'eps_animado_gen',
 'eps_animado_rf',
 'eps_acolhedor_ani',
 'eps_rispido_aco']

In [10]:
data_p = '/l/disk1/awstebas/data/TTS/speaker-adriana/'

In [15]:
## CHECKING FOR WAV22 FOLDER IN EACH STYLE
c = True
t = True
for e in eps_paths:
    if('wav22' not in os.listdir(data_p + e)):
        print("There is no 'wav22' folder in ", e)
        c = False
    if('transcricao' not in os.listdir(data_p + e)):
        print("There is no 'transcricao' folder in ", e)
        t = False
        
if(c == True):
    print("All folders have wav22 file")

if(t == True):
    print("All folders have transcricao file")

All folders have wav22 file
All folders have transcricao file


In [21]:
## CHECKING TEXT STRUCTURE

ex = '/l/disk1/awstebas/data/TTS/speaker-adriana/eps_rispido_aco/transcricao/eps_rispido_aco.txt'

with open(ex, 'r') as f:
    for line in f.readlines():
        print(line.split(':'))
        break

['eps_rispido_aco0001', ' A pergunta que resta é se houve ou não suborno.\n']


In [26]:
## defining get_wav and text

styles = []
wavs = []
texts = []
for e in eps_paths:
    local_dir = data_p + e
    
    transc_path = local_dir + '/transcricao/' + e + '.txt'
    with open(transc_path, 'r') as f:
        for line in f.readlines():
            splitted = line.split(':')
            
            wpath = local_dir + '/wav22/' + splitted[0] + '.wav'
           
            if(os.path.isfile(wpath)):
                wavs.append(wpath)

                texts.append(splitted[1][1:-1])

                style = e.split('_')[1]
                styles.append(style)
            else:
                print(f'{wpath} is not a file')

In [28]:
len(texts)

5180

In [29]:
def get_phones(txt):
    
    with open('tmp.txt', "w", encoding="utf-8") as f:
        f.writelines(txt)
    
    cmd = '/workspace/tool_language/tool_language -l /workspace/tool_language/libptbr.so.4.6.0 -p /workspace/tool_language/ -i tmp.txt -o tmp_pnh.txt --phonemes -s'
    
    os.system(cmd)
    
    with open ('tmp_pnh.txt', "r") as f:
        lines = f.readlines()
    
    return lines[0][:-1]

In [30]:
get_phones(texts[0])

"#p 'oo ij kk aa rf ll 'an wn #s bb 'on dz 'ii ac #e kk ic bb 'on ff aa ll 'aa rf kk on vv oo ss 'ee nn oh vv aa mm 'en ts ic #p"

In [31]:
phones = []

for t in texts:
    phones.append(get_phones(t))

In [46]:
speakers = ['adriana'] * len(phones)
df = pd.DataFrame({'norm_text': texts,
                   'phonetic_transcription': phones,
                   'wav_path': wavs,
                   'style': styles,
                   'speaker': speakers})

df.head(), df.shape

(                                           norm_text  \
 0  Oi Carlão; bom dia! Que bom falar com você nov...   
 1  Claro que podemos ajudá-la Melissa. Estamos aq...   
 2  Você ligou para o lugar certo Martín. Vamos ti...   
 3  A Orelha de Van Gogh é um conjunto de música e...   
 4  Se você quiser, eu passo pela sua casa hoje de...   
 
                               phonetic_transcription  \
 0  #p 'oo ij kk aa rf ll 'an wn #s bb 'on dz 'ii ...   
 1  #p kk ll 'aa rr uc kk ic pp oo dd 'ee mm uc zz...   
 2  #p vv oo ss 'ee ll ii gg 'oo uw pp 'aa rr ac u...   
 3  #p ac oo rr 'ee lh ac dz ic vv 'an gg 'oh gg i...   
 4  #p ss ic vv oo ss 'ee kk ii zz 'eh rf #c 'ee u...   
 
                                             wav_path   style  speaker  
 0  /l/disk1/awstebas/data/TTS/speaker-adriana/eps...  neutro  adriana  
 1  /l/disk1/awstebas/data/TTS/speaker-adriana/eps...  neutro  adriana  
 2  /l/disk1/awstebas/data/TTS/speaker-adriana/eps...  neutro  adriana  
 3  /l/disk1/awsteba

In [47]:
df['phonetic_transcription'].values[:3]

array(["#p 'oo ij kk aa rf ll 'an wn #s bb 'on dz 'ii ac #e kk ic bb 'on ff aa ll 'aa rf kk on vv oo ss 'ee nn oh vv aa mm 'en ts ic #p",
       "#p kk ll 'aa rr uc kk ic pp oo dd 'ee mm uc zz aa zh uu dd 'aa ll ac mm ee ll 'ii ss ac #p ee ss tt 'an mm uc zz aa kk 'ii pp rd ac 'ii ss uc #p",
       "#p vv oo ss 'ee ll ii gg 'oo uw pp 'aa rr ac uc ll uu gg 'aa rf ss 'eh rf tt uc mm aa rf ts 'in #p vv 'an mm uc ss ts ii rr 'aa rf tt 'oo dd ac zz ac ss ss 'uu ac zz dd 'uu vv ic dd ac ss #p"],
      dtype=object)

# Now lets particionate in train, val and test...

Where, test set will be 50% paired samples among all styles and the other 50% will be non-paired samples

In [48]:
df['style'].value_counts()

neutro       1309
acolhedor    1308
animado      1307
rispido      1256
Name: style, dtype: int64

In [72]:
# Lets get a flag in which we have same phonemes

gp = df.groupby('phonetic_transcription').count().reset_index().sort_values(by='speaker', ascending= False)
gp['len'] = gp.phonetic_transcription.str.len()
gp.head(30)

,phonetic_transcription,norm_text,wav_path,style,speaker,len
123,#p 'ee uw ss 'oh ee ss tt 'oo uw tt en tt 'an ...,8,8,8,8,73
422,#p ac kk 'uu uw pp ac ff 'oo ij dd uc ss ss ee...,5,5,5,5,168
239,#p 'oh lh ac #c mm 'ee zz mm uc kk ic pp rd ee...,5,5,5,5,250
87,#p 'ee uw ee ss tt 'oo uw mm 'un in tt uc bb r...,5,5,5,5,165
1053,#p mm 'aa zz 'aa uw ll 'aa dd uc dd ac aa dd o...,5,5,5,5,246
638,#p ee ss pp 'eh rr uc kk ic ac ss kk 'oo ij zz...,5,5,5,5,118
1337,#p pp 'oh dz ic 'ii rf ss ic kk ii zz 'eh rf #...,5,5,5,5,165
714,#p en tt 'an wn vv 'an mm uc ss kk oo mm ee ss...,5,5,5,5,75
1292,#p pp 'aa rr ac 'ii ss uc #c vv 'oo uw mm ic a...,4,4,4,4,283
1293,#p pp 'aa rr ac aa tt en dd 'ee rr ac dd ee mm...,4,4,4,4,90


In [74]:
df[df['phonetic_transcription'] == gp.phonetic_transcription.values[2]]

,norm_text,phonetic_transcription,wav_path,style,speaker
232,"Olha, mesmo que precisemos recomeçar a operaçã...",#p 'oh lh ac #c mm 'ee zz mm uc kk ic pp rd ee...,/l/disk1/awstebas/data/TTS/speaker-adriana/eps...,rispido,adriana
353,"Olha, mesmo que precisemos recomeçar a operaçã...",#p 'oh lh ac #c mm 'ee zz mm uc kk ic pp rd ee...,/l/disk1/awstebas/data/TTS/speaker-adriana/eps...,rispido,adriana
657,"Olha, mesmo que precisemos recomeçar a operaçã...",#p 'oh lh ac #c mm 'ee zz mm uc kk ic pp rd ee...,/l/disk1/awstebas/data/TTS/speaker-adriana/eps...,neutro,adriana
1843,"Olha, mesmo que precisemos recomeçar a operaçã...",#p 'oh lh ac #c mm 'ee zz mm uc kk ic pp rd ee...,/l/disk1/awstebas/data/TTS/speaker-adriana/eps...,acolhedor,adriana
3403,"Olha, mesmo que precisemos recomeçar a operaçã...",#p 'oh lh ac #c mm 'ee zz mm uc kk ic pp rd ee...,/l/disk1/awstebas/data/TTS/speaker-adriana/eps...,animado,adriana


In [77]:
import IPython
f =df[df['phonetic_transcription'] == gp.phonetic_transcription.values[2]]['wav_path'].values[-1]
IPython.display.Audio(f)

In [66]:
import IPython
f =df[df['phonetic_transcription'] == gp.phonetic_transcription.values[0]]['wav_path'].values[4]
IPython.display.Audio(f)

In [80]:
50/5180

0.009652509652509652

# Lets get 20 paired and 30 unpaired samples

Because, in 20 paired samples we have at least 80 audio samples (1 for each 4 styles_), while in 30 unpaired samples we have only 30 audio samples.

In [83]:
# Lets get a flag in which we have same phonemes

gp = df.groupby('phonetic_transcription').count().reset_index().sort_values(by='speaker', ascending= False)
gp['len'] = gp.phonetic_transcription.str.len()

paired = gp.head(20).phonetic_transcription.values
paired

array(["#p 'ee uw ss 'oh ee ss tt 'oo uw tt en tt 'an dd uc aa zh uu dd 'aa rf #p",
       "#p ac kk 'uu uw pp ac ff 'oo ij dd uc ss ss ee gg uu rr 'an ss ac ss #c kk ic aa zh 'ii rr an wn kk 'oo mm uc vv ee rf dd aa dd 'ee ij rr uc zz aa nn ii mm 'aa ij ss #p",
       "#p 'oh lh ac #c mm 'ee zz mm uc kk ic pp rd ee ss ii zz 'ee mm uc zz rx ee kk oo mm ee ss 'aa rr ac oo pp ee rr aa ss 'an wn #c vv 'an mm uc ss ff aa zz 'ee rr 'ii ss uc ic rx ee zz oo uw vv 'ee rf dz ic vv 'ee zz uc ss pp rd oo bb ll 'ee mm ac ss #p",
       "#p 'ee uw ee ss tt 'oo uw mm 'un in tt uc bb rd 'aa vv ac kk on vv oo ss 'ee #c en tt 'an wn nn 'an wn ff 'aa ll ic kk oo mm 'ii gg uc pp oo rr en kk uw 'an tt uc #p",
       "#p mm 'aa zz 'aa uw ll 'aa dd uc dd ac aa dd oo ll ee ss 'en ts ic en kk on tt rd 'an mm uc zz un pp aa ss aa pp 'oh rf ts ic kk on uc nn 'oh mm ic dz ic 'uu mm ac pp ee ss 'oo ac kk ic pp oo dd ee rr 'ii ac ss 'ee rf ss 'ee uw ii rf mm 'an wn #p",
       "#p ee ss pp 'eh rr uc kk ic ac ss k

In [85]:
# Lets get a flag in which we have same phonemes

gp = df.groupby('phonetic_transcription').count().reset_index().sort_values(by='speaker', ascending= True)
gp['len'] = gp.phonetic_transcription.str.len()

nonpaired = gp[gp['speaker'] == 1].phonetic_transcription.values
len(nonpaired)

663

In [87]:
npfilt = df[df.phonetic_transcription.isin(nonpaired)]

In [89]:
nonpaired_neutral = npfilt[npfilt['style'] == 'neutro'].sample(5, random_state = 42).phonetic_transcription.values
nonpaired_neutral

array(["#p ee ss tt 'an mm uc zz oo uw vv 'in dd uc un bb aa rr 'uu lh uc #c ac tt 'eh ss tt ac kk aa ij pp 'ii rr ac vv 'aa ij tt 'ee rf kk uw aa dd rd 'ii lh ac #p",
       "#p tt 'en nh uc bb 'oo ac zz nn oo ts 'ii ss ij ac ss pp 'aa rr ac vv oo ss 'ee #s dd 'oo nn ac ss uu zz 'an nn ac #p",
       "#p pp aa rr 'eh ss ic aa tt 'eh ff 'aa ss ic uw en tt en dd 'ee rf #c zh oo gg 'aa rf ss in pp ll ii zz mm 'en ts ic 'uu mm ac bb 'oh ll ac dz ic tt 'ee nn ic zz nn ac pp aa rr 'ee dz ic ic ff in zh 'ii rf kk ic ac rx 'uu ac en ff rd 'en ts ic ac ss 'uu ac kk 'aa zz ac mm uu dd 'oo uw dz ic kk 'oo rf #p",
       "#p ss 'en pp rd ic ss on nh 'oo uw tt 'ee rr un mm uu zz 'ee uw en pp 'oo rf tt uc aa ll 'eh gg rd ic #p",
       "#p ic ee ss pp 'eh rr uc kk ic kk on ts ii nn 'uu en ss 'en pp rd ic aa ss 'in #c kk 'oo mm uc nn ac ff 'oh tt uc #c ic kk ic tt 'uu dd uc dd 'ee ss 'eh rf tt uc pp rd ac vv oo ss 'ee ss #p"],
      dtype=object)

In [90]:
nonpaired_animado = npfilt[npfilt['style'] == 'animado'].sample(5, random_state = 42).phonetic_transcription.values
nonpaired_animado

array(["#p 'ii ss uc mm 'ee zz mm uc #e mm 'aa zz ll 'en bb rd ic ss ic kk ic 'ee ss ts ic ss 'uu pp ee rf dd ee ss kk 'on tt uc 'eh vv 'aa ll ic dd uc pp 'aa rr ac pp aa gg aa mm 'en tt uc 'aa vv 'ii ss tt ac ic aa pp 'ee nn ac zz en 'uu mm ac dz ic nn 'oh ss ac zz ll 'oh zh ac zz dd uc sh 'oh pp in #p",
       "#p pp 'ee ll uc aa mm 'oo rf dz ic dd 'ee uw ss #c mm ee nn 'oh rf kk ic pp ac bb 'aa rx ac #c mm aa ij 'oh rf kk ic mm ic ee ss kk 'uu ts ic #p",
       "#p uc kk ic 'eh un aa bb ss 'uu rf dd uc #c pp 'oo ij ss ss ic uc nn 'oh ss uc pp ll an nn 'ee tt ac dd ee ij sh 'aa rf dz ic ee zz ii ss ts 'ii rf #c nn 'an wn aa vv ee rr 'aa uc kk ic 'oo uw kk 'oo mm uc kk on ss uu mm 'ii rf #p",
       "#p ss 'in tt uc kk ic 'ee uw tt 'en nh uc kk ic gg an nh 'aa rr 'ii ss uc #c mm 'aa ss kk uw 'an dd uc aa kk aa bb 'aa rf vv 'oo uw ss en ts 'ii rf ff 'aa uw tt ac #e",
       "#p pp oo dd 'ee mm uc zz aa zh uu dd 'aa rf vv oo ss 'ee #c mm 'aa ss pp 'aa rr ac 'ii ss uc pp rd ee ss ii zz '

In [91]:
nonpaired_acolhedor = npfilt[npfilt['style'] == 'acolhedor'].sample(5, random_state = 42).phonetic_transcription.values
nonpaired_acolhedor

array(["#p uc 'aa bb ic tt uc dz ic bb rd in kk 'aa rf vv 'en dz ic pp ee kk 'ee nn uc kk 'oo mm uc kk on ff 'ii rf mm ac ac mm 'an jn #c mm aa rr 'ii ac aa pp aa rr ee ss 'ii dd ac #p",
       "#p tt an bb 'en ee mm ii ts 'ii rr an wn un aa ll 'eh rf tt ac dz ic vv ii 'aa zh en nn uc ff 'in dd uc 'an nn uc nn uc mm 'ee zz rx ee tt rd aa zz 'aa dd uc #p",
       "#p 'ee uw pp 'oh ss uc ss oo ll ii ss ii tt 'aa rr aa gg 'oh rr ac uc dd ee zz bb ll oo kk 'ee ij uw #c mm 'aa zz nn 'an wn ee ss kk 'ee ss ac kk ic pp rd ee ss 'ii zz ac pp aa gg 'aa rr en aa tt 'eh vv 'in ts ic ic kk uw 'aa tt rd uc 'oh rr ac ss tt 'oo dd ac zz ac ss ff aa tt 'uu rr ac zz vv en ss 'ii dd ac ss pp 'aa rr ac ee vv ii tt 'aa rf kk ic 'eh ll ac ss 'ee zh ac bb ll oo kk ee 'aa dd ac nn oh vv aa mm 'en ts ic #p",
       "#p nn 'an wn ff 'aa uw tt ac tt 'an tt uc #s 'eh ss 'oh ee ss pp ee rr 'aa rr aa tt 'eh ac dd ee ff 'ee zz ac dd ac tt 'eh zz ic pp 'aa rr ac kk on ss ee gg 'ii rr ac pp rd oo mm oo ss 'an wn nn u

In [92]:
nonpaired_rispido = npfilt[npfilt['style'] == 'rispido'].sample(5, random_state = 42).phonetic_transcription.values
nonpaired_rispido

array(["#p aa pp 'oh zz 'uu mm ac bb rd 'eh vv ic vv 'aa ij ac ac ff 'ii ll ac ss ic dz ii ss ii pp 'oo uw ic aa pp 'ee nn ac zz aa uw gg 'uu mm ac ss pp ee ss 'oo ac ss pp ee rf mm aa nn ee ss 'ee rr an wn nn uc ll oo kk 'aa uw #p",
       "#p uc ff ll 'aa vv ic uw ss 'aa bb ic kk ic ff 'oo ij rx 'uu bb en ss kk ic pp ee gg 'oo uw ss 'ee uw ss sh oo kk oo ll 'aa ts ic zz dd uu rr 'an ts ic ac kk uw aa rr en tt 'ee nn ac #p",
       "#p 'eh rr ac mm 'aa ij ss kk ic 'uu mm ac kk on ss ee kk uw 'en ss ij ac ff aa tt 'aa uw dd uc nn 'oh ss uc mm 'eh tt oo dd uc dz ic kk oo ll oo nn ii zz aa ss 'an wn en tt 'eh rx ac aa bb un dd 'an ts ic ic dd ee ss pp oo vv oo 'aa dd ac #p",
       "#p aa tt 'eh un dz 'ii ac #p mm 'in nh ac aa mm 'ii gg ac #p",
       "#p kk ic rx ee 'uu nn ic uc ss pp aa 'ii zz ic zz dd ee zz en vv oo uw vv 'ii dd uc ss ss en dd ee ij sh 'aa rr uc ss pp aa 'ii zz ic zz ee mm ee rf zh 'en ts ic ss #p"],
      dtype=object)

In [98]:
nonpaired_neutral = list(nonpaired_neutral)
nonpaired_animado = list(nonpaired_animado)
nonpaired_acolhedor = list(nonpaired_acolhedor)
nonpaired_rispido = list(nonpaired_rispido)

nonpaired = []
nonpaired.extend(nonpaired_neutral)
nonpaired.extend(nonpaired_animado)
nonpaired.extend(nonpaired_rispido)
nonpaired.extend(nonpaired_acolhedor)

paired = list(paired)
len(nonpaired), len(paired)

(20, 20)

In [109]:
# Lets just hear some examples if its all right
import IPython
f =df[df['phonetic_transcription'] == nonpaired_neutral[4]]['wav_path'].values[0]
IPython.display.Audio(f)

In [114]:
# Lets just hear some examples if its all right
import IPython
f =df[df['phonetic_transcription'] == nonpaired_animado[0]]['wav_path'].values[0]
IPython.display.Audio(f)

In [119]:
# Lets just hear some examples if its all right
import IPython
f =df[df['phonetic_transcription'] == nonpaired_rispido[0]]['wav_path'].values[0]
IPython.display.Audio(f)

In [124]:
# Lets just hear some examples if its all right
import IPython
f =df[df['phonetic_transcription'] == nonpaired_acolhedor[0]]['wav_path'].values[0]
IPython.display.Audio(f)

Aparentemente tudo certo, vamos agora splitar o conjuntos de teste inteiro


In [125]:
df_test_paired = df[df.phonetic_transcription.isin(paired)]
df_test_paired.shape

(91, 5)

In [128]:
df_test_paired.value_counts('style')

style
rispido      28
acolhedor    21
animado      21
neutro       21
dtype: int64

In [127]:
df_test_unpaired = df[df.phonetic_transcription.isin(nonpaired)]
df_test_unpaired.shape

(20, 5)

In [129]:
# Finally, lets save files as meta_data format for cpqd_read, which use ';' as separator

cols = ['phonetic_transcription', 'wav_path', 'speaker','style']

df_test_paired[cols].to_csv('styles_paired_test.csv', index = False, sep=';', encoding = 'utf-8')
df_test_unpaired[cols].to_csv('styles_unpaired_test.csv', index = False, sep=';', encoding = 'utf-8')

# Now lets define our "rest" dataset and split in train and val randomly

In [131]:
total_to_exclude = []
total_to_exclude.extend(nonpaired)
total_to_exclude.extend(paired)

df_rest = df[~df.phonetic_transcription.isin(total_to_exclude)]
df_rest.shape, df.shape

((5069, 5), (5180, 5))

In [132]:
from sklearn.model_selection import train_test_split

In [138]:
df_train, df_val = train_test_split(df_rest, test_size = 0.03, stratify=df_rest['style'], random_state = 42)

In [139]:
import librosa

def get_durations(wpath):
    y, sr = librosa.load(wpath, sr = None)
    return len(y)/sr

In [140]:
durs = []
for w in df_train['wav_path']:
    durs.append(get_durations(w))
df_train['durs'] = durs

durs = []
for w in df_val['wav_path']:
    durs.append(get_durations(w))
df_val['durs'] = durs

In [141]:
df_train.durs.sum()/3600, df_val.durs.sum()/3600

(6.39019417989418, 0.20501904761904763)

In [142]:
# df_train per speaker
df_train.groupby('style').agg({'durs': 'sum'})/3600

,durs
style,
acolhedor,1.637522
animado,1.589717
neutro,1.638228
rispido,1.524726


In [143]:
# df_val per speaker
df_val.groupby('style').agg({'durs': 'sum'})/3600

,durs
style,
acolhedor,0.059894
animado,0.050144
neutro,0.047675
rispido,0.047306


# Now, getting the train and val dataset from universal list and getting only 3 speakers data

In [144]:
u_train = pd.read_csv("universal_cpqd_train.csv", encoding= 'utf-8', sep=';')
u_val = pd.read_csv("universal_cpqd_val.csv", encoding= 'utf-8', sep=';')

In [146]:
u_train['style'] = 'neutro'
u_val['style'] = 'neutro'

speakers_in = ['adriana','chiquinho','rosana']

u_train = u_train[u_train['speaker'].isin(speakers_in)]
u_val = u_val[u_val['speaker'].isin(speakers_in)]

u_train.speaker.unique(), u_val.speaker.unique()

(array(['adriana', 'chiquinho', 'rosana'], dtype=object),
 array(['adriana', 'chiquinho', 'rosana'], dtype=object))

In [147]:
def get_rec(wpath):
    return wpath.split('/')[7]

recs = []
for w in u_train.wav_path:
    recs.append(get_rec(w))
u_train['rec'] = recs

recs = []
for w in u_val.wav_path:
    recs.append(get_rec(w))
u_val['rec'] = recs

<ipython-input-147-865d679dc8fb>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  u_train['rec'] = recs


In [149]:
u_train.rec.unique(), u_val.rec.unique()

(array(['aec', 'akiva', 'akiva_fidelizacao', 'akiva_portabilidade',
        'aplicacao', 'atendimento', 'avm', 'avm_custom', 'bradesco',
        'bradesco_bia', 'bradesco_ura', 'callcenter', 'chat_covid',
        'comemorativas', 'datas', 'digitos', 'dnk_expr',
        'elev_dentalprime', 'emocao', 'empresas', 'enderecos',
        'freqpalavras', 'horas', 'ibm', 'ibm_fastshop', 'ibm_tim',
        'interrogativas', 'interrogativas_ou', 'interrogativas_qu',
        'interrogativas_sn', 'invenit', 'marketing_cpqd', 'neutro',
        'nomes', 'nomes_freq', 'nomes_int', 'numeros', 'pg_amc',
        'pg_americo', 'pg_avon', 'pg_demoKami_v2', 'pg_demoPrioritario',
        'pg_flex', 'pg_generico', 'pg_interrogativas', 'pg_localcred',
        'pg_meucartao', 'pg_renner', 'pg_riachuelo',
        'pg_riachuelo_expressivo', 'pg_vivo', 'quantias', 'renaut',
        'riqueza_fonetica', 'robbu', 'saudacoes', 'siglas',
        'sulamerica_auto', 'sulamerica_saude', 'sulamerica_vida',
        'textofa

In [153]:
u_train.shape , u_train[u_train.rec.isin(['rf_selecionadas'])].shape

((22807, 5), (1905, 5))

In [165]:
u_train[(u_train.rec.isin(['riqueza_fonetica']))].speaker.value_counts()

adriana      2157
chiquinho    1870
Name: speaker, dtype: int64

In [171]:
u_train[(u_train['speaker'].isin(['rosana','chiquinho'])) | ((u_train['speaker'] == 'adriana') & (u_train['rec'] == 'riqueza_fonetica'))].speaker.value_counts()

rosana       6684
chiquinho    4680
adriana      2157
Name: speaker, dtype: int64

In [172]:
u_train = u_train[(u_train['speaker'].isin(['rosana','chiquinho'])) | ((u_train['speaker'] == 'adriana') & (u_train['rec'] == 'riqueza_fonetica'))]

In [160]:
u_val.speaker.value_counts()

adriana      108
rosana        50
chiquinho     30
Name: speaker, dtype: int64

In [175]:
## Getting only the necessary cols
cols = ['phonetic_transcription', 'wav_path', 'speaker','style']

u_train = u_train[cols]
u_val = u_val[cols]

df_train = df_train[cols]
df_val = df_val[cols]

df_train_tot = pd.concat([df_train, u_train]).reset_index(drop = True)
df_val_tot = pd.concat([df_val, u_val]).reset_index(drop = True)

df_train_tot.shape, df_val_tot.shape

((18437, 4), (341, 4))

In [176]:
df_train_tot.head()

,phonetic_transcription,wav_path,speaker,style
0,#p kk uu uw ts ii vv 'aa rr un bb uu zz 'ee ij...,/l/disk1/awstebas/data/TTS/speaker-adriana/eps...,adriana,animado
1,#p bb uu rr 'aa kk uc nn ac 'aa rf vv oo rr ic...,/l/disk1/awstebas/data/TTS/speaker-adriana/eps...,adriana,acolhedor
2,#p aa kk rd ee dz 'ii tt uc kk ic ss 'ee zh ac...,/l/disk1/awstebas/data/TTS/speaker-adriana/eps...,adriana,animado
3,#p uc bb aa rr 'uu lh uc dd ac rx 'uu ac nn 'a...,/l/disk1/awstebas/data/TTS/speaker-adriana/eps...,adriana,neutro
4,#p aa kk 'ee ll ic ss kk ic dz 'ii zz en kk ic...,/l/disk1/awstebas/data/TTS/speaker-adriana/eps...,adriana,neutro


In [177]:
df_train_tot.speaker.value_counts()

adriana      7073
rosana       6684
chiquinho    4680
Name: speaker, dtype: int64

In [178]:
df_val_tot.speaker.value_counts()

adriana      261
rosana        50
chiquinho     30
Name: speaker, dtype: int64

In [179]:
durs = []
for w in df_train_tot['wav_path']:
    durs.append(get_durations(w))
df_train_tot['durs'] = durs

durs = []
for w in df_val_tot['wav_path']:
    durs.append(get_durations(w))
df_val_tot['durs'] = durs

In [181]:
df_train_tot.durs.sum()/3600, df_val_tot.durs.sum()/3600

(20.32140468631897, 0.42365879314688837)

In [182]:
# df_train per speaker
df_train_tot.groupby('style').agg({'durs': 'sum'})/3600

,durs
style,
acolhedor,1.637522
animado,1.589717
neutro,15.569439
rispido,1.524726


In [183]:
# df_train per speaker
df_train_tot.groupby('speaker').agg({'durs': 'sum'})/3600

,durs
speaker,
adriana,8.543069
chiquinho,4.551724
rosana,7.226612


In [184]:
cols = ['phonetic_transcription', 'wav_path', 'speaker','style']

df_train_tot[cols].to_csv('mp_styles_train.csv', index = False, sep=';', encoding = 'utf-8')
df_val_tot[cols].to_csv('mp_styles_val.csv', index = False, sep=';', encoding = 'utf-8')